# Physics-Informed Neural Networs

Теоретическая основа: универсальная теорема аппроксимации - теорема Хорника-Цирельсона-Уайта: \
Однослойная нейроная сеть с достаточно большим числом нейронов и нелинейной гладкой функцией \
(например, sigmoid или tanh) может приблизить любую непрерывную функцию на компактном множестве \
с любой заданной точностью. 

$$
sup_{x \in K}| f(x) - N(x) | < \epsilon
$$

# Уравнение теплопроводности

$$
\frac{\sigma u(x,t)}{\sigma t}=\frac{\sigma}{\sigma x}(k(x)*\frac{\sigma u(x,t)}{\sigma x})+f(x,t))
$$

$ u(x,t) $ - температура  \
$ k(x) $ \
$ f(x,t) $

# аппроксимация дифференциального уравнения нейроной сетью
$ N_u(x,t) $ - нейросеть аппроксимирует u(x,t), \
$ N_k(x) $ - нейросеть аппроксимирует k(x), \
тогда аппроксимированное ур-ие теплопроводностипримет вид:

$$
\frac{\sigma N_u(x,t)}{\sigma t}=\frac{\sigma}{\sigma x}(N_k(x)*\frac{\sigma N_u(x,t)}{\sigma x})+f(x,t))
$$

или раскроем производную произведения и перенесем все в левую сторону
$$
\frac{\sigma N_u(x,t)}{\sigma t} - \frac{\sigma N_k(x)}{\sigma x}*\frac{\sigma N_u(x,t)}{\sigma x} - N_k(x)*\frac{\sigma^2 N_u(x,t)}{\sigma x^2} -f(x,t)) = 0
$$

Как выглядит производная нейросети
Пусть $ N_u(x,t) $ - это гладкая функция как композиция слоев нейросети \
$$ N_u(x,t) = f_3(f_2(f_1(x,t,\theta), \theta), \theta), $$ \
$ f_1, f_2, f_3 $ - слои нейросети (линейные преобразования + функции активации) \
$ \theta_i $ - обучаемые параметры (веса и смещения) \
производная $  N_u(x,t) $ вычисляется по цепному правилу:
$$
\frac{\sigma N_u}{\sigma x} = \frac{\sigma f_3}{\sigma f_2} * \frac{\sigma f_2}{\sigma f_1} * \frac{\sigma f_1}{\sigma x}
$$
где каждый член это якобиан (матрица частных производных)

# функция потерь (Loss Function)
### 1. Data loss: ошибка предсказанных и эксперементальных данных

$$
L_{data} = \frac{1}{N} \sum_{i=1}^N | N_u(x_i,t_i) - u_{data}(x_i, t_i)|^2
$$

### 2. Physics loss: ошибка аппроксимации дифф.ур-ия нейноной сетью

$$
L_{phys} = \frac{1}{M} \sum_{j=1}^M | \frac{\sigma N_u(x,t)}{\sigma t} - \frac{\sigma N_k(x)}{\sigma x}*\frac{\sigma N_u(x,t)}{\sigma x} - N_k(x)*\frac{\sigma^2 N_u(x,t)}{\sigma x^2} -f(x,t)) |^2
$$

Итоговый лосс:
$$
L = L_{data} + L_{phys}
$$

Пусть для упрощения $ N_k(x) = const=1 $ и $ f(x,t)=0 $

# Создаем нейроную сеть:
### 2 (вход) -> 5 (1-ый внутр.слой) -> 5 (2-ой внутр.слой) -> 1 (выход)

In [172]:
# Создаем нейросеть 2->5->5->1
import torch
import torch.nn as nn

class Unet( nn.Module ):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(2,5),
            nn.Sigmoid(),
            nn.Linear(5,5),
            nn.Sigmoid(),
            nn.Linear(5,1)
        )

    def forward(self, x, t):
        return self.fc( torch.cat([x, t], dim=1) ) # self.layers()
    

In [173]:
u_net = Unet()
u_net

Unet(
  (fc): Sequential(
    (0): Linear(in_features=2, out_features=5, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [174]:
x = torch.tensor([.1, .2])
x

tensor([0.1000, 0.2000])

# Проверка работы нейросети (от входа на выход)
### 2 нейрона (вход) -> 5 нейронов (1-ый внутр.слой)
$$
h_i = Sigmoid(\sum_{j=1}^2 w_{ij}x_j + b_i), i = 1,2,3,4,5
$$

или в матричном кратком виде: $ h = Sigmoid(W*x+b) $, или в матричном полном виде:
$$
\begin{vmatrix}
h_1 \\
h_2 \\
h_3 \\
h_4 \\
h_5
\end{vmatrix} \quad = \quad
Sigmoid
\begin{pmatrix}
\begin{vmatrix}
w_{11} & w_{12} \\
w_{21} & w_{22} \\
w_{31} & w_{32} \\
w_{41} & w_{42} \\
w_{51} & w_{52}
\end{vmatrix} *
\begin{vmatrix}
x_1 \\
x_2
\end{vmatrix} \quad + \quad
\begin{vmatrix}
b_1 \\
b_2 \\
b_3 \\
b_4 \\
b_5
\end{vmatrix}
\end{pmatrix}
$$

In [175]:
# 2-> 5
print(u_net.fc[0].weight)
print(u_net.fc[0].bias)

Parameter containing:
tensor([[ 0.1113,  0.3221],
        [ 0.2452,  0.6260],
        [-0.5157, -0.5329],
        [-0.1988,  0.3053],
        [-0.5104, -0.4744]], requires_grad=True)
Parameter containing:
tensor([-0.3232, -0.4728,  0.0057,  0.2650, -0.2076], requires_grad=True)


In [176]:
# h = Sigmoid( W(1)x+b(1) )
h = torch.sigmoid( u_net.fc[0].weight @ x + u_net.fc[0].bias )
h

tensor([0.4384, 0.4199, 0.4620, 0.5759, 0.4125], grad_fn=<SigmoidBackward0>)

### 5 нейрона (1-ый внутр.слой) -> 5 нейронов (2-ый внутр.слой)
$$
g_i = Sigmoid(\sum_{j=1}^5 w_{ij}h_j + b_i), i = 1,2,3,4,5
$$

или в матричном кратком виде: $ g = Sigmoid(W*h+b) $, или в матричном полном виде:
$$
\begin{vmatrix}
g_1 \\
g_2 \\
g_3 \\
g_4 \\
g_5
\end{vmatrix} \quad = \quad
Sigmoid
\begin{pmatrix}
\begin{vmatrix}
w_{11} & w_{12} & w_{13} & w_{14} & w_{15} \\
w_{21} & w_{22} & w_{23} & w_{24} & w_{25} \\
w_{31} & w_{32} & w_{33} & w_{34} & w_{35} \\
w_{41} & w_{42} & w_{43} & w_{44} & w_{45} \\
w_{51} & w_{52} & w_{53} & w_{54} & w_{55} 
\end{vmatrix} *
\begin{vmatrix}
h_1 \\
h_2 \\
h_3 \\
h_4 \\
h_5
\end{vmatrix} \quad + \quad
\begin{vmatrix}
b_1 \\
b_2 \\
b_3 \\
b_4 \\
b_5
\end{vmatrix}
\end{pmatrix}
$$

In [177]:
# 5-> 5
print(u_net.fc[2].weight)
print(u_net.fc[2].bias)

Parameter containing:
tensor([[ 0.1593, -0.1711,  0.2269, -0.1217,  0.0942],
        [ 0.3273, -0.2330,  0.0509, -0.0731, -0.1502],
        [-0.0316, -0.0227, -0.3228, -0.0854,  0.3469],
        [-0.1750,  0.0991, -0.0585,  0.3064, -0.2058],
        [ 0.2415, -0.2975, -0.3738, -0.0868,  0.3571]], requires_grad=True)
Parameter containing:
tensor([ 0.2358,  0.1530,  0.0451, -0.3772, -0.4041], requires_grad=True)


In [178]:
# g = Sigmoid( W(2)h+b(2) )
g = torch.sigmoid( u_net.fc[2].weight @ h + u_net.fc[2].bias )
g

tensor([0.5762, 0.5295, 0.4916, 0.4139, 0.3779], grad_fn=<SigmoidBackward0>)

### 5 нейрона (2-ый внутр.слой) -> 1 нейронов (выход)
$$
y = \sum_{j=1}^1 w_{ij}h_j + b_i, i = 1,2,3,4,5
$$

или в матричном кратком виде: $ y = W*g+b $, или в матричном полном виде:
$$
\begin{vmatrix}
y_1
\end{vmatrix} \quad = \quad
\begin{vmatrix}
w_{11} & w_{12} & w_{13} & w_{14} & w_{15} \\
\end{vmatrix} *
\begin{vmatrix}
g_1 \\
g_2 \\
g_3 \\
g_4 \\
g_5
\end{vmatrix} \quad + \quad
\begin{vmatrix}
b_1
\end{vmatrix}
$$

In [179]:
# 5 -> 1
print(u_net.fc[4].weight)
print(u_net.fc[4].bias)

Parameter containing:
tensor([[-0.2682,  0.3943,  0.0843, -0.3182,  0.1998]], requires_grad=True)
Parameter containing:
tensor([0.0281], requires_grad=True)


In [180]:
# y = W(3)g+b(3)
y = u_net.fc[4].weight @ g + u_net.fc[4].bias
y

tensor([0.0676], grad_fn=<AddBackward0>)

# Проверка вычислений

In [181]:
xv = torch.tensor([[.1],[1.]])
tv = torch.tensor([[.2],[2.]])
print(torch.cat([xv,tv], dim=1))
y_check = u_net(xv,tv)
y_check

tensor([[0.1000, 0.2000],
        [1.0000, 2.0000]])


tensor([[0.0676],
        [0.0659]], grad_fn=<AddmmBackward0>)

In [182]:
if y == y_check[0]:
    print("OK")
else:
    print("FAILED")



OK


In [156]:
L=1.0
T=0.1
x = torch.rand(10, 1, requires_grad=True )*L
t = torch.rand(10, 1, requires_grad=True )*T
torch.cat([x,t], dim=1)

tensor([[0.5287, 0.0171],
        [0.3403, 0.0095],
        [0.2443, 0.0507],
        [0.0586, 0.0707],
        [0.7441, 0.0175],
        [0.5478, 0.0367],
        [0.2898, 0.0916],
        [0.3746, 0.0289],
        [0.3580, 0.0481],
        [0.7775, 0.0632]], grad_fn=<CatBackward0>)

In [141]:
# предсказания u
u_pred = u_net(x,t)
u_pred

tensor([[0.1497],
        [0.1480],
        [0.1489],
        [0.1476],
        [0.1481],
        [0.1469],
        [0.1492],
        [0.1489],
        [0.1475],
        [0.1472]], grad_fn=<AddmmBackward0>)

In [142]:
print(u_net.fc[0].weight)
print(u_net.fc[0].bias)

Parameter containing:
tensor([[ 0.5721, -0.4791],
        [-0.2451, -0.2255],
        [-0.0775, -0.6565],
        [ 0.2754, -0.5267],
        [-0.2110, -0.6138]], requires_grad=True)
Parameter containing:
tensor([ 0.5869, -0.6385, -0.6661, -0.0092, -0.4487], requires_grad=True)


In [143]:
sig = nn.Sigmoid()
sig(torch.tensor([[1.0, 2.0],[3., 4.]]))

tensor([[0.7311, 0.8808],
        [0.9526, 0.9820]])

In [144]:
optimizer = torch.optim.Adam( u_net.parameters(), lr=1e-3 )
optimizer.zero_grad()

In [145]:
du_dt = torch.autograd.grad(u_pred, t, grad_outputs=torch.ones_like(t), create_graph=True)[0]
du_dt

/home/kimve/.local/lib/python3.10/site-packages/torch/autograd/graph.py:824: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


tensor([[-0.0115],
        [-0.0118],
        [-0.0118],
        [-0.0121],
        [-0.0121],
        [-0.0121],
        [-0.0113],
        [-0.0114],
        [-0.0120],
        [-0.0122]], grad_fn=<SliceBackward0>)

In [146]:
du_dx = torch.autograd.grad(u_pred, x, grad_outputs=torch.ones_like(x), create_graph=True)[0]
du_dx

tensor([[0.0022],
        [0.0025],
        [0.0024],
        [0.0027],
        [0.0026],
        [0.0027],
        [0.0022],
        [0.0022],
        [0.0026],
        [0.0027]], grad_fn=<SliceBackward0>)

In [147]:
d2u_dx2 = torch.autograd.grad(du_dx, x, grad_outputs=torch.ones_like(x), create_graph=True)[0]
d2u_dx2

tensor([[-0.0008],
        [-0.0007],
        [-0.0007],
        [-0.0005],
        [-0.0006],
        [-0.0005],
        [-0.0008],
        [-0.0008],
        [-0.0006],
        [-0.0005]], grad_fn=<SliceBackward0>)

$$
L_{phys} = \frac{1}{M} \sum_{j=1}^M | \frac{\sigma N_u(x,t)}{\sigma t} - \frac{\sigma^2 N_u(x,t)}{\sigma x^2} |^2
$$

In [148]:
loss_phys = torch.mean(du_dt - d2u_dx2)**2
loss_phys

tensor(0.0001, grad_fn=<PowBackward0>)

In [149]:
optimizer.zero_grad()

In [150]:
loss_phys.backward()
loss_phys

tensor(0.0001, grad_fn=<PowBackward0>)

In [151]:
optimizer.step()

In [152]:
loss_phys

tensor(0.0001, grad_fn=<PowBackward0>)

In [153]:
print(u_net.fc[0].weight)
print(u_net.fc[0].bias)

Parameter containing:
tensor([[ 0.5731, -0.4781],
        [-0.2441, -0.2245],
        [-0.0785, -0.6555],
        [ 0.2764, -0.5257],
        [-0.2100, -0.6128]], requires_grad=True)
Parameter containing:
tensor([ 0.5879, -0.6395, -0.6671, -0.0082, -0.4497], requires_grad=True)
